In [1]:
from io import StringIO
from pdfminer import pdfpage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageFont, ImageDraw
import camelot

filename='DUGJY'
PDF = f'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/{filename}.pdf'
pd.set_option('display.max_colwidth', None)

In [2]:

def make_df_image(table,
                  max_cols=-1,
                  max_rows=-1):
    """Return dataframe as image."""
    with tempfile.NamedTemporaryFile(suffix='.jpg',
                                     delete=False) as tmp:
        dfi.export(table, tmp.name, max_cols=max_cols, max_rows=max_rows)
        image = mpimg.imread(tmp.name)
        return image


def make_lines_image(lines,
                     mrgn=15,
                     background=(255, 255, 255),
                     text_color=(0, 0, 0),
                     font_size=10):
    """Return raw text in lines as image."""
    lines = pd.Series(lines)
    longest_line = lines[lines.str.len().idxmax()]
    image = Image.new("RGBA", (1, 1))
    draw = ImageDraw.Draw(image)
    monospace = ImageFont.truetype(r'С:\Windows\Fonts\cour.ttf', font_size)
    line_width, line_height = draw.textsize(longest_line, monospace)
    img_width, img_height = (line_width + mrgn * 2,
                             len(lines) * line_height + mrgn * 2)
    image = Image.new("RGBA", (img_width, img_height), background)
    draw = ImageDraw.Draw(image)
    x, y0 = (mrgn, mrgn)
    for n, line in enumerate(lines):
        y = y0 + n * line_height
        draw.text((x, y), line, text_color, monospace)
    return image


In [59]:
def extract(extract_no):
    pdfminer_string = StringIO()
    pageno = 0
    with open(PDF, "rb") as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr,
                            pdfminer_string,
                            laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            if pageno == extract_no:
                interpreter.process_page(page)
            pageno = pageno+1
    pdfminer_lines = pdfminer_string.getvalue().splitlines()
    pdfminer_lines = [ln for ln in pdfminer_lines if ln]
    return pdfminer_lines

def camel(extract_no):
    tables, text = camelot.read_pdf(PDF, strip_text='\n', pages=str(extract_no))
    tables.export(r'C:\Users\33669\PycharmProjects\OCR\outputs\table.html', f='html', compress=True) # json, excel, html, markdown, sqlite
    tablesfin = []
    for table in text:
        para = []
        for row_index,row in enumerate(table):
            for col_index,col in enumerate(row):
                if row_index != 0:
                    line = f'{table[0][col_index]} {col}'
                    #print(line)
                    para.append(line)
        tabel = ' '.join(para)
        tablesfin.append(tabel)
    for table in tablesfin:
        print(table)
                       

In [60]:
def extracted(pdfminer_lines):
    # pd.Series(pdfminer_lines)
    fig, ax = plt.subplots(figsize=(50,25))
    title = 'pdfminer.six 20201018'
    for i,img in enumerate(map(make_lines_image,
                                [pdfminer_lines])):
        ax.axis("off")
        ax.set_adjustable("box")
        ax.title.set_text(title)
        ax.imshow(img)
    return fig.tight_layout()

In [62]:
pagen = 4
#lines = extract(pagen)
camel(pagen)
#extracted(lines)

Sr.No  Designation  Complete Bunched Cables   Overall dia approx  Total mass approx Sr.No  Designation  Complete Bunched Cables (mm)  (kg/km.) Sr.No 1. Designation 3x50+16+35 Complete Bunched Cables 32  640 Sr.No 2. Designation 3x70+16+50 Complete Bunched Cables 34  890 Sr.No 3. Designation 3x95+16+70 Complete Bunched Cables 39  1180 Sr.No 4. Designation 3x120+16+95 Complete Bunched Cables 42  1430 Sr.No 5. Designation 3x150+16+120 Complete Bunched Cables 49  1905
